In [1]:
import zarr
import os

path = '/hpc/vvib872/Data/PSF/CV42-us'
name = 'uS-fine_640_000_0.zarr'
store = zarr.NestedDirectoryStore(os.path.join(path,name))
z = zarr.open(store, mode='r')
# z= zarr.zeros((128,7424,8704),chunks=z_r.chunks, dtype = '|b')
# z[:] = z_r[-128:, :, :]
z.info

Type,zarr.core.Array
Data type,float32
Shape,"(880, 2048, 17408)"
Chunk shape,"(1, 512, 512)"
Order,C
Read-only,True
Compressor,"Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)"
Store type,zarr.storage.NestedDirectoryStore
No. bytes,125493575680 (116.9G)
No. bytes stored,368
Storage ratio,341015151.3


In [3]:
import matplotlib.pyplot as plt

img = z[:,:,:]
# plt.imshow(img[0,:,:])

In [4]:
import SimpleITK as sitk

itkimage = sitk.Cast(sitk.GetImageFromArray(img[:,:,:]),sitk.sitkFloat32)
itkimage.SetSpacing((1.0, 1.0, 1.0))
sitk.WriteImage(itkimage,os.path.join(path, name + '.tif') , True) 

In [4]:
import SimpleITK as sitk
from skimage.measure import label, regionprops
import numpy as np

img_scale = 255
output_dir = '/hpc_utoz/vvib872/Data/deeplearning_data/3D/Train/border_del_new_label'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
def save_Tiffs(img,index):
    
    if np.sum(img) > 80000:
        label_img = label(img, connectivity=3)             
        rprop = regionprops(label_img)

#         for r in rprop:
#             if r.area < 5000 and r.area > 0:            
#                 img[r.coords.T[0,:],r.coords.T[1,:],r.coords.T[2,:]] = 0
            
        itkimage = sitk.Cast(sitk.GetImageFromArray(img_scale *img),sitk.sitkUInt8)
        itkimage.SetSpacing((1.0, 1.0, 1.0))
        sitk.WriteImage(itkimage,os.path.join(output_dir, "chunk_%05d.tif" % index) , True)    
        return  np.atleast_3d(1)
    else:
        return  np.atleast_3d(0)
    
def cube_deletion(img,index):
        if np.sum(img) > 80000:
            label_img = label(img, connectivity=3)             
            rprop = regionprops(label_img)

            for r in rprop:
                if r.area < 5000 and r.area > 0:                    
                    img[r.coords.T[0,:],r.coords.T[1,:],r.coords.T[2,:]] = 0
                    
            [z,x,y] = np.random.randint(low=32,high= 96, size=3)
                    
            img[z-16:z+16,x-16:x+16,y-16:y+16] = 0            
            itkimage = sitk.Cast(sitk.GetImageFromArray(img_scale *img),sitk.sitkUInt8)
            itkimage.SetSpacing((1.0, 1.0, 1.0))
            sitk.WriteImage(itkimage,os.path.join(output_dir, "cd_chunk_%05d.tif" % index) , True)    
            return  np.atleast_3d(1)
        else:
            return  np.atleast_3d(0)   
    
def middle_deletion(img,index):
#         if np.sum(img) > 80000:
#             label_img = label(img, connectivity=3)             
#             rprop = regionprops(label_img)

#             for r in rprop:
#                 if r.area < 5000 and r.area > 0:                    
#                     img[r.coords.T[0,:],r.coords.T[1,:],r.coords.T[2,:]] = 0
                    
            img[64-5:64+5,:,:] = 0            
            itkimage = sitk.Cast(sitk.GetImageFromArray(img_scale *img),sitk.sitkUInt8)
            itkimage.SetSpacing((1.0, 1.0, 1.0))
            sitk.WriteImage(itkimage,os.path.join(output_dir, "md_chunk_%05d.tif" % index) , True)            
            return  np.atleast_3d(1)
#         else:
#             return  np.atleast_3d(0) 
        
def border_deletion(img,index):
        if np.sum(img) > 80000:
            label_img = label(img, connectivity=3)             
            rprop = regionprops(label_img)

            for r in rprop:
                if r.area < 5000 and r.area > 0:                    
                    img[r.coords.T[0,:],r.coords.T[1,:],r.coords.T[2,:]] = 0
                    
            img[64-5:64+5,:,:] = 0            
            itkimage = sitk.Cast(sitk.GetImageFromArray(img_scale *img),sitk.sitkUInt8)
            itkimage.SetSpacing((1.0, 1.0, 1.0))
            sitk.WriteImage(itkimage,os.path.join(output_dir, "md_chunk_%05d.tif" % index) , True)            
            return  np.atleast_3d(1)
        else:
            return  np.atleast_3d(0)

da_out = da.map_blocks(save_Tiffs, da_input, da_index, chunks=(1,1,1), dtype='b')
da_out

,Array,Chunk
Bytes,22.27 kB,1 B
Shape,"(6, 32, 116)","(1, 1, 1)"
Count,66818 Tasks,22272 Chunks
Type,int8,numpy.ndarray


In [5]:
from dask.distributed import Client
client = Client()
client.cluster

In [48]:
da_out.compute()
a=1

In [9]:
import os
from sklearn.model_selection import train_test_split

x_filenames = [os.path.join(output_dir, f) for f in sorted(os.listdir(output_dir))]

x_train_filenames, x_test_filenames = train_test_split(x_filenames, test_size=0.2, random_state=42)

In [8]:
import shutil
import os

test_folder = '/hpc_utoz/vvib872/Data/deeplearning_data/3D/Test/not_cleaned_new_label'

if not os.path.exists(test_folder):
    os.makedirs(test_folder)

for f in x_test_filenames:    
        shutil.move(f, test_folder)   